# Catalyst Cooperative Jupyter Notebook Template
This notebook lays out a standard format and some best practices for creating interactive / exploratory notebooks which can be relatively easily shared between different PUDL users, and turned into reusable Python modules for integration into our underlying Python packages.

## Begin with a narrative outline
Each notebook should start with a brief explanation (in Markdown) explaining the purpose of the analysis, and outlining the different stages / steps which are taken to accomplish the analysis.
As the analysis develops, you can add new sections or details to this section.

## Notebooks should be runnable
Insofar as it's possible, another PUDL user who has cloned the repository that the notebook is part of should be able to update their `pudl-dev` conda environment, open the notebook, and run all cells successfully.
If there are required data or other prerequisites that the notebook cannot manage on its own -- like a file that needs to be downloaded by hand and placed in a particular location -- those steps should be laid out clearly at the beginning of the notebook.

## Avoid Troublesome Elements

### Don't hardcode passwords or access tokens
Most of our work is done in public Github repositories.
No authentication information should ever appear in a notebook.
These values can be stored in environment variables on your local computer.

### Do not hardocde values, especially late in the notebook
If the analysis depends on particular choices of input values, those should be called out explicitly at the beginning of the notebook.
(NB: We should explore ways to parameterize notebooks, [papermill](https://papermill.readthedocs.io/en/latest/) is one tool that does this).

### Don't hardcode absolute file paths
If anyone is going to be able to use the notebook, the files it uses will need to be stored somewhere that makes sense on both your and other computers.
We assume that anyone using this template has the PUDL package installed, and has a local PUDL data management environment set up.
  * Input data that needs to be stored on disk and accessed via a shared location should be saved under `<PUDL_IN>/data/local/<data_source>/`.
  * Intermediate saved data products (e.g. a pickled result of a computationally intensive process) and results should be saved to a location relative to the notebook, and within the directory hierarchy of the repository that the notebook is part of.
  
### Don't require avoidable long-running computations
Consider persisting to disk the results of computations that take more than a few minutes, if the outputs are small enough to be checked into the repository and shared with other users.
Only require the expensive computation to be run if this pre-computed output is not available.

### Don't litter
Don't leave lots of additional code laying around, even commented out, "just in case" you want to look at it again later.
Notebooks need to be relatively linear in the end, even though the thought processes and exploratory analyses that generate them may not be.
Once you have a working analysis, either prune those branches, or encapsulate them as options within functions.

### Don't load unneccesary libraries
Only import libraries which are required by the notebook, to avoid unnecessary dependencies.
If your analysis requires a new library that isn't yet part of the shared `pudl-dev` environment, add it to the `devtools/environment.yml` file so that others will get it when they update their environment.

## Related Resources:
Lots of these guidelines are taken directly from Emily Riederer's post: [RMarkdown Driven Development](https://emilyriederer.netlify.app/post/rmarkdown-driven-development/).
For more in depth explanation of the motivations behind this layout, do go check it out!

# Import Libraries
* Because it's very likely that you will be editing the PUDL Python packages or your own local module under development while working in the notebook, use the %autoreload magic with autoreload level 2 to ensure that any changes you've made in those files are always reflected in the code that's running in the notebook.
* Put all import statements at the top of the notebook, so everyone can see what its dependencies are up front, and so that if an import is going to fail, it fails early, before the rest of the notebook is run.
* Try to avoid importing individual functions and classes from deep within packages, as it may not be clear to other users where those elements came from, later in the notebook, and also to minimize the chance of namespace collisions.

In [1]:
%load_ext autoreload
%autoreload 2

In [683]:
# Standard libraries
import logging
import os
import pathlib
import sys

# 3rd party libraries
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
import sqlalchemy as sa

# Local libraries
import pudl
import pudl.constants as pc

# Configure Display Parameters

In [684]:
sns.set()
%matplotlib inline
mpl.rcParams['figure.figsize'] = (10,4)
mpl.rcParams['figure.dpi'] = 150
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

# Use Python Logging facilities
* Using a logger from the beginning will make the transition into the PUDL package easier.
* Creating a logging handler here will also allow you to see the logging output coming from PUDL and other underlying packages.

In [685]:
logger=logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

# Define Functions
In many cases, the eventual product of a notebook analysis is going to be the creation of new, reusable functions that are integrated into the underlying PUDL code. You should begin the process of accumulating and organizing those functions as soon as you notice repeated patterns in your code.
* Functions should be used to encapsulate any potentially reusable code.
* Functions should be defined immediately after the imports, to avoid accidental dependence on zombie variables that are defined further down in the code
* While they may evolve over time, having brief docstrings explaining what they do will help others understand them.
* If there's a particular type of plot or visualization that is made repeatedly in the notebook, it's good to parameterize and functionalize it here too, so that as you refine the presentation of the data and results, those improvements can be made in a single place, and shown uniformly throughout the notebook.
* As these functions mature and become more general purpose tools, you will probably want to start migrating them into their own local module, under a `src` directory in the same directory as the notebook. You will want to import this module 

## Dummy EIA 861 ETL

In [686]:
def test_etl_eia(eia_inputs, pudl_settings):
    """
    This is a dummy function that runs the first part of the EIA ETL
    process -- everything up until the entity harvesting begins. For
    use in this notebook only.

    """
    eia860_tables = eia_inputs["eia860_tables"]
    eia860_years = eia_inputs["eia860_years"]
    eia861_tables = eia_inputs["eia861_tables"]
    eia861_years = eia_inputs["eia861_years"]
    eia923_tables = eia_inputs["eia923_tables"]
    eia923_years = eia_inputs["eia923_years"]

    # generate CSVs for the static EIA tables, return the list of tables
    #static_tables = _load_static_tables_eia(datapkg_dir)

    # Extract EIA forms 923, 860
    eia860_raw_dfs = pudl.extract.eia860.Extractor().extract(eia860_years, testing=True)
    eia861_raw_dfs = pudl.extract.eia861.Extractor().extract(eia861_years, testing=True)
    eia923_raw_dfs = pudl.extract.eia923.Extractor().extract(eia923_years, testing=True)

    # Transform EIA forms 860, 861, 923
    eia860_transformed_dfs = pudl.transform.eia860.transform(eia860_raw_dfs, eia860_tables=eia860_tables)
    eia861_transformed_dfs = pudl.transform.eia861.transform(eia861_raw_dfs, eia861_tables=eia861_tables)
    eia923_transformed_dfs = pudl.transform.eia923.transform(eia923_raw_dfs, eia923_tables=eia923_tables)

    # create an eia transformed dfs dictionary
    eia_transformed_dfs = eia860_transformed_dfs.copy()
    eia_transformed_dfs.update(eia861_transformed_dfs.copy())
    eia_transformed_dfs.update(eia923_transformed_dfs.copy())

    # convert types..
    eia_transformed_dfs = pudl.helpers.convert_dfs_dict_dtypes(eia_transformed_dfs, 'eia')

    return eia860_raw_dfs, eia861_raw_dfs, eia923_raw_dfs, eia_transformed_dfs

# Define Notebook Parameters
If there are overarching parameters which determine the nature of the analysis -- which US states to look at, which utilities are of interest, a particular start and end date -- state those clearly at the beginning of the analysis, so that they can be referred to by the rest of the notebook and easily changed if need be.
* If the analysis depends on local (non-PUDL managed) datasets, define the paths to those data here.
* If there are external URLs or other resource locations that will be accessed, define those here as well.
* This is also where you should create your `pudl_settings` dictionary and connections to your local PUDL databases

In [687]:
pudl_settings = pudl.workspace.setup.get_defaults()
display(pudl_settings)

ferc1_engine = sa.create_engine(pudl_settings['ferc1_db'])
display(ferc1_engine)

pudl_engine = sa.create_engine(pudl_settings['pudl_db'])
display(pudl_engine)

# Is there other external data you need to pull in?
# If so, put it in a (relatively) standard place on the filesystem.
my_new_data_url = "https://mynewdata.website.gov/path/to/new/data.csv"
my_new_datadir = pathlib.Path(pudl_settings["data_dir"]) / "local/new_data_source"

# Store API keys and other secrets in environment variables
# and read them in here, if needed:
# API_KEY_EIA = os.environ["API_KEY_EIA "]
# API_KEY_FRED = os.environ["API_KEY_FRED "]

{'pudl_in': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR',
 'data_dir': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/data',
 'settings_dir': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/settings',
 'pudl_out': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR',
 'sqlite_dir': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/sqlite',
 'parquet_dir': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/parquet',
 'datapkg_dir': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/datapkg',
 'notebook_dir': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/notebook',
 'ferc1_db': 'sqlite:////Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/sqlite/ferc1.sqlite',
 'pudl_db': 'sqlite:////Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/sqlite/pudl.sqlite'}

Engine(sqlite:////Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/sqlite/ferc1.sqlite)

Engine(sqlite:////Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/sqlite/pudl.sqlite)

# Load Data
* Given the above parameters and functions, it should now be possible to load the required data into local variables for further wrangling, analysis, and visualization.
* If the data is not yet present on the machine of the person running the notebook, this step should also acquire the data from its original source, and place it in the appropriate location under `<PUDL_IN>/data/local/`.
* If there are steps which have to be done manually here, put them first so that they fail first if the user hasn't read the instructions, and they can fix the situation before a bunch of other work gets done. Try to minimize the amount of work in the filesystem that has to be done manually though.
* If this process takes a little while, don't be shy about producing `logging` output.
* Using the `%%time` cell magic can also help users understand which pieces of work / data acquisition are hard:

## EIA 861 (2010-2018)
* Not yet fully integrated into PUDL
* Post-transform harvesting process isn't compatible w/ EIA 861 structure
* Only getting the `sales_eia861`, `balancing_authority_eia861`, and `service_territory_eia861` tables

### Already Transformed EIA 861 DataFrames

In [688]:
%%time
eia_years = list(range(2010, 2019))
eia_inputs = {
    "eia860_years": [],
    "eia860_tables": pudl.constants.pudl_tables["eia860"],
    "eia861_years": eia_years,
    "eia861_tables": pudl.constants.pudl_tables["eia861"],
    "eia923_years": [],
    "eia923_tables": pudl.constants.pudl_tables["eia923"],
}
eia860_raw_dfs, eia861_raw_dfs, eia923_raw_dfs, eia_transformed_dfs = test_etl_eia(eia_inputs=eia_inputs, pudl_settings=pudl_settings)

No years given. Not extracting eia860 spreadsheet data.
Loading dataframe for eia861 advanced_metering_infrastructure_eia861 2010
Loading dataframe for eia861 advanced_metering_infrastructure_eia861 2011
Loading dataframe for eia861 advanced_metering_infrastructure_eia861 2012
Loading dataframe for eia861 advanced_metering_infrastructure_eia861 2013
Loading dataframe for eia861 advanced_metering_infrastructure_eia861 2014
Loading dataframe for eia861 advanced_metering_infrastructure_eia861 2015
Loading dataframe for eia861 advanced_metering_infrastructure_eia861 2016
Loading dataframe for eia861 advanced_metering_infrastructure_eia861 2017
Loading dataframe for eia861 advanced_metering_infrastructure_eia861 2018
Loading dataframe for eia861 balancing_authority_eia861 2010
Loading dataframe for eia861 balancing_authority_eia861 2011
Loading dataframe for eia861 balancing_authority_eia861 2012
Loading dataframe for eia861 balancing_authority_eia861 2013
Loading dataframe for eia861 balan

# Sanity Check Data
If there's any validation that can be done on the data which you've loaded to flag if/when it is inappropriate for the analysis that follows, do it here. If you find the data is unusable, use `assert` statements or `raise` Exceptions to stop the notebook from proceeding, and indicate what the problem is.

In [689]:
list(eia861_raw_dfs.keys())

['advanced_metering_infrastructure_eia861',
 'balancing_authority_eia861',
 'demand_response_eia861',
 'demand_side_management_eia861',
 'distributed_generation_eia861',
 'distribution_systems_eia861',
 'dynamic_pricing_eia861',
 'frame_eia861',
 'green_pricing_eia861',
 'mergers_eia861',
 'net_metering_eia861',
 'non_net_metering_eia861',
 'operational_data_eia861',
 'reliability_eia861',
 'sales_eia861',
 'service_territory_eia861',
 'short_form_eia861',
 'utility_data_eia861']

In [690]:
list(eia_transformed_dfs.keys())

['service_territory_eia861',
 'balancing_authority_eia861',
 'sales_eia861',
 'demand_response_eia861']

In [693]:
eia_transformed_dfs['demand_response_eia861']
#13205 -- without BA
#13225 -- with BA

,utility_id_eia,state,balancing_authority_code_eia,utility_name_eia,water_heater,customer_class,actual_peak_demand_savings_mw,customer_incentives_cost,customers,energy_savings_mwh,other_costs,potential_peak_demand_savings_mw,report_date
0,84,VA,UNK,A & N Electric Coop,<NA>,commercial,0.0,NaN,0,0.0,NaN,0.0,2013-01-01
1,84,VA,UNK,A & N Electric Coop,<NA>,industrial,0.0,NaN,0,0.0,NaN,0.0,2013-01-01
2,84,VA,UNK,A & N Electric Coop,<NA>,residential,1.4,21000.0,2200,100.0,NaN,1.4,2013-01-01
4,84,VA,UNK,A & N Electric Coop,<NA>,transportation,NaN,NaN,<NA>,NaN,NaN,NaN,2013-01-01
5,97,IL,UNK,Adams Electric Coop,91,commercial,NaN,NaN,60,NaN,NaN,4.0,2013-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13299,60482,OK,SWPP,CKenergy Electric Cooperative,<NA>,transportation,0.0,NaN,0,NaN,0.0,0.0,2018-01-01
13300,60631,MI,MISO,Upper Michigan Energy Resources Corp.,<NA>,commercial,NaN,NaN,1,NaN,NaN,0.0,2018-01-01
13301,60631,MI,MISO,Upper Michigan Energy Resources Corp.,<NA>,industrial,NaN,NaN,6,NaN,NaN,19.9,2018-01-01
13302,60631,MI,MISO,Upper Michigan Energy Resources Corp.,<NA>,residential,NaN,NaN,74,NaN,NaN,0.1,2018-01-01


# Preliminary Data Wrangling
Once all of the data is loaded and looks like it's in good shape, do any initial wrangling that's specific to this particular analysis. This should mostly make use of the higher level functions which were defined above. If this step takes a while, don't be shy about producing `logging` outputs.

In [613]:
df = eia_transformed_dfs['demand_response_eia861']

idx_cols = [
    "utility_id_eia",
    "state",
    "report_date",
    "balancing_authority_code_eia",
]

val_cols = [
    'actual_peak_demand_savings_mw',
    'customer_incentives_cost',
    'customers',
    'energy_savings_mwh',
    'other_costs',
    'potential_peak_demand_savings_mw',
]

In [631]:
def compare_totals(df, idx_cols, val_cols):
    df_no_tot = df.query("customer_class!='total'")
    df_tot = df.query("customer_class=='total'")
    # observed = true helps with categorical dtypes so they dont replicate for every single index value
    df_sum = df_no_tot.groupby(idx_cols, observed=True).sum().reset_index()
    df_merge = pd.merge(df_sum, df_tot, on=idx_cols, how='outer', suffixes=('_sum', '_tot'))
    
    # Update ROUND value and BOOL indicator value.
    # TRUE means that difference between columns is negligable / they are the same.
#     for col in val_cols: 
#         df_merge[col+'_sum'] = df_merge.apply(lambda x: x[col+'_sum'].round(2))
#         df_merge[col+'_bool'] = abs(df_merge[col+'_sum'] - df_merge[col+'_tot']) < 0.2
#         if df_merge[col+'_sum'] == 0:
#             df_merge[col+'_bool'] = False
    return df_merge 

In [640]:
mer = compare_totals(df, idx_cols, val_cols)

In [642]:

#mer['actual_peak_demand_savings_mw_sum']

In [647]:
col = 'actual_peak_demand_savings_mw'
df.assign({col+'_sum': lambda x: x[col+'_sum'].round(2)})

TypeError: assign() takes 1 positional argument but 2 were given

## Demand Response

In [648]:
dr = eia_transformed_dfs['demand_response_eia861']

In [649]:
#dr.groupby(['utility_id_eia', 'balancing_authority_code_eia', 'report_date']).sum()
#13225
dr.groupby(['utility_id_eia','balancing_authority_code_eia']).sum()
#dr.loc[dr['state']=='AK']
#dr['balancing_authority_code_eia'].describe()
dr.loc[dr['utility_id_eia']==84]
dr['balancing_authority_code_eia']

0         UNK
1         UNK
2         UNK
3         UNK
4         UNK
         ... 
13300    MISO
13301    MISO
13302    MISO
13303    MISO
13304    MISO
Name: balancing_authority_code_eia, Length: 13225, dtype: string

In [654]:
idx_cols = [
    "utility_id_eia",
    "state",
    "balancing_authority_code_eia",
    "report_date",
]

val_cols = [
    'actual_peak_demand_savings_mw',
     'customer_incentives_cost',
     'customers',
     'energy_savings_mwh',
     'other_costs',
     'potential_peak_demand_savings_mw',
]

In [655]:
#dr_piv = dr.pivot_table(val_cols, idx_cols, 'customer_class')

In [663]:
dr_no_tot = dr.query("customer_class!='total'")
dr_tot = dr.query("customer_class=='total'")
#dr_tot = dr_tot.drop(['bal', 'utility_name_eia'], axis=1)
dr_sum = dr_no_tot.groupby(idx_cols, observed=True).sum().reset_index()

In [664]:
df = pd.merge(dr_sum, dr_tot, on=idx_cols, how='outer', suffixes=('_sum', '_tot'))
df = df.set_index(idx_cols)

In [665]:
df['actual_peak_demand_savings_mw_sum'] = df['actual_peak_demand_savings_mw_sum'].round(2)
df['customer_incentives_cost_sum'] = df['customer_incentives_cost_sum'].round(2)
#df['customers_sum'] = df['customers_sum'].round(2)
df['energy_savings_mwh_sum'] = df['energy_savings_mwh_sum'].round(2)
df['other_costs_sum'] = df['other_costs_sum'].round(2)
df['potential_peak_demand_savings_mw_sum'] = df['potential_peak_demand_savings_mw_sum'].round(2)

In [666]:
for col in val_cols:
    df[col+'_bool'] = abs(df[col+'_sum'] - df[col+'_tot']) < 0.2

In [667]:
# df['actual_peak_demand_savings_mw_bool'].describe() # 2641/2645 TRUE -- 40211, IN, 2017, 2018
# df['customer_incentives_cost_bool'].describe() # 2613/2645 TRUE -- ALL zeros
# df['customers_bool'].describe() # 2498/2502 TRUE -- ALL zeros
# df['energy_savings_mwh_bool'].describe() # 2615/2645 -- ALL zeros
# df['other_costs_bool'].describe() # 2605/2645 -- ALL zeros
# df['potential_peak_demand_savings_mw_bool'].describe() # 2641/2645 -- 40211, IN, 2017, 2018

In [679]:
test = df.query("energy_savings_mwh_bool==False")
t = test.query("energy_savings_mwh_sum!=0")
#test[['potential_peak_demand_savings_mw_sum', 'potential_peak_demand_savings_mw_tot']]
#(t['potential_peak_demand_savings_mw_sum'] - t['potential_peak_demand_savings_mw_tot'])
t = t.reset_index()

In [680]:

#1367	WI	2018-01-01 ----- 33.33% (other costs), 10% (customer_incentives),
#6342	AR	2018-01-01 ----- 10% (other costs)
# 11581	IA	 2017 ----- 33.333% (energy savings)
# 13725	NE 2018 ----- 25% (energy savings)
# 14132	IA 2018 ----- 25% (energy savings)


In [682]:
t = t.assign(pct=lambda x: 1 / x.energy_savings_mwh_tot * 100)
#df2 = df.reset_index()
t

,utility_id_eia,state,balancing_authority_code_eia,report_date,water_heater_sum,actual_peak_demand_savings_mw_sum,customer_incentives_cost_sum,customers_sum,energy_savings_mwh_sum,other_costs_sum,potential_peak_demand_savings_mw_sum,utility_name_eia,water_heater_tot,customer_class,actual_peak_demand_savings_mw_tot,customer_incentives_cost_tot,customers_tot,energy_savings_mwh_tot,other_costs_tot,potential_peak_demand_savings_mw_tot,actual_peak_demand_savings_mw_bool,customer_incentives_cost_bool,customers_bool,energy_savings_mwh_bool,other_costs_bool,potential_peak_demand_savings_mw_bool,pct
0,195,AL,SOCO,2017-01-01,0,945.5,0.0,6277,2032.0,24329000.0,1519.4,Alabama Power Co,<NA>,total,945.5,0.0,6277,2031.0,24328000.0,1519.4,True,True,True,False,False,True,0.049237
1,965,IA,MISO,2017-01-01,0,0.6,9000.0,360,39.0,5000.0,0.6,Atlantic Municipal Utilities,<NA>,total,0.6,9000.0,360,38.0,5000.0,0.6,True,True,True,False,True,True,2.631579
2,1009,MN,MISO,2017-01-01,0,4.1,18000.0,6631,197.0,0.0,4.1,City of Austin - (MN),<NA>,total,4.1,18000.0,6631,196.0,NaN,4.1,True,True,True,False,False,True,0.510204
3,3249,NY,NYIS,2018-01-01,0,9.5,0.0,3274,66.0,0.0,9.5,Central Hudson Gas & Elec Corp,<NA>,total,9.5,NaN,3274,65.0,NaN,9.5,True,False,True,False,False,True,1.538462
4,4117,VA,PJM,2018-01-01,0,8.2,65000.0,1603,25.0,0.0,10.0,Community Electric Coop,<NA>,total,8.2,65000.0,1603,24.0,NaN,10.0,True,True,True,False,False,True,4.166667
5,5056,IA,SWPP,2017-01-01,0,1.8,106000.0,2629,29.0,0.0,1.8,City of Denison - (IA),<NA>,total,1.8,106000.0,2629,30.0,NaN,1.8,True,True,True,False,False,True,3.333333
6,5056,IA,SWPP,2018-01-01,0,1.7,108000.0,2612,29.0,0.0,1.7,City of Denison - (IA),<NA>,total,1.8,108000.0,2612,30.0,NaN,1.8,True,True,True,False,False,True,3.333333
7,5701,TX,EPE,2018-01-01,0,11.4,607000.0,1982,68.0,27000.0,11.4,El Paso Electric Co,<NA>,total,11.4,607000.0,1982,67.0,27000.0,11.4,True,True,True,False,True,True,1.492537
8,6151,MN,MISO,2018-01-01,0,0.7,4000.0,3939,61.0,1000.0,1.7,Fairmont Public Utilities Comm,0,total,0.7,4000.0,3939,62.0,1000.0,1.7,True,True,True,False,True,True,1.612903
9,6411,GA,SOCO,2018-01-01,0,10.0,434000.0,23069,261.0,274000.0,54.0,Flint Electric Membership Corp,<NA>,total,10.0,434000.0,23069,260.0,274000.0,54.0,True,True,True,False,True,True,0.384615


In [520]:
df2.loc[df2['utility_id_eia']==40211]

,utility_id_eia,state,report_date,water_heater_sum,actual_peak_demand_savings_mw_sum,customer_incentives_cost_sum,customers_sum,energy_savings_mwh_sum,other_costs_sum,potential_peak_demand_savings_mw_sum,balancing_authority_code_eia,utility_name_eia,water_heater_tot,customer_class,actual_peak_demand_savings_mw_tot,customer_incentives_cost_tot,customers_tot,energy_savings_mwh_tot,other_costs_tot,potential_peak_demand_savings_mw_tot,actual_peak_demand_savings_mw_bool,customer_incentives_cost_bool,customers_bool,energy_savings_mwh_bool,other_costs_bool,potential_peak_demand_savings_mw_bool,energy_savings_COMP
2591,40211,IL,2013-01-01,0,9.5,434000.0,5270,0.0,856000.0,9.5,UNK,"Wabash Valley Power Assn, Inc",<NA>,total,9.5,434000.0,5270,NaN,856000.0,9.5,True,True,True,False,True,True,False
2592,40211,IL,2014-01-01,2928,8.6,443000.0,5270,34.0,401000.0,9.4,UNK,"Wabash Valley Power Assn, Inc",732,total,8.6,443000.0,5270,34.0,401000.0,9.4,True,True,True,True,True,True,True
2593,40211,IL,2015-01-01,2352,0.0,486000.0,4949,0.0,263000.0,10.2,UNK,"Wabash Valley Power Assn, Inc",588,total,NaN,486000.0,4949,NaN,263000.0,10.2,False,True,True,False,True,True,False
2594,40211,IL,2016-01-01,0,8.6,482000.0,4962,0.0,293000.0,10.0,UNK,"Wabash Valley Power Assn, Inc",<NA>,total,8.6,482000.0,4962,NaN,293000.0,10.0,True,True,True,False,True,True,False
2595,40211,IL,2017-01-01,0,8.1,464000.0,4625,0.0,0.0,9.5,MISO,"Wabash Valley Power Assn, Inc",<NA>,total,8.1,464000.0,4625,0.0,NaN,9.5,True,True,True,True,False,True,True
2596,40211,IL,2018-01-01,0,8.4,490000.0,4562,0.0,0.0,9.4,MISO,"Wabash Valley Power Assn, Inc",<NA>,total,8.4,490000.0,4562,0.0,NaN,9.4,True,True,True,True,False,True,True
2597,40211,IN,2017-01-01,0,38.4,2202000.0,21648,0.0,10000.0,45.3,PJM,"Wabash Valley Power Assn, Inc",<NA>,total,2.5,152000.0,84,NaN,1000.0,3.1,False,False,False,False,False,False,False
2598,40211,IN,2017-01-01,0,38.4,2202000.0,21648,0.0,10000.0,45.3,MISO,"Wabash Valley Power Assn, Inc",<NA>,total,35.9,2050000.0,21564,NaN,9000.0,42.2,False,False,False,False,False,False,False
2599,40211,IN,2018-01-01,0,40.5,2359000.0,21948,0.0,0.0,45.4,PJM,"Wabash Valley Power Assn, Inc",<NA>,total,2.8,168000.0,88,0.0,0.0,3.2,False,False,False,True,True,False,True
2600,40211,IN,2018-01-01,0,40.5,2359000.0,21948,0.0,0.0,45.4,MISO,"Wabash Valley Power Assn, Inc",<NA>,total,37.7,2191000.0,21860,0.0,0.0,42.2,False,False,False,True,True,False,True


In [492]:
tt = dr.loc[(dr['utility_id_eia']==195) & (dr['state']=='AL')].tail(10).reset_index()
tt['energy_savings_mwh'][3]


2031.0

In [293]:
#dr_group = dr_no_tot.groupby(idx_cols).sum().reset_index()

In [296]:
dr_no_tot.groupby('utility_id_eia').sum()

,water_heater,actual_peak_demand_savings_mw,customer_incentives_cost,customers,energy_savings_mwh,other_costs,potential_peak_demand_savings_mw
utility_id_eia,,,,,,,
84,0,8.6,39000.0,14024,597.0,67000.0,18.0
97,1012,23.5,1127000.0,25953,0.0,48000.0,50.0
108,392,89.0,943000.0,29940,76680.0,1034000.0,474.8
162,0,0.2,0.0,214,0.0,32000.0,0.4
189,38088,82.7,0.0,96386,0.0,2548000.0,82.7
...,...,...,...,...,...,...,...
58126,0,0.0,0.0,0,0.0,0.0,0.0
58127,0,0.0,104000.0,65,0.0,348000.0,0.0
58128,0,7.9,6063000.0,1792,52284.0,18536000.0,0.0


In [297]:
test = dr_group.loc[dr_group['actual_peak_demand_savings_mw'].notna()]

In [298]:
dr_84 = dr.loc[(dr['utility_id_eia']==84)]
dr_84.query("report_date==2017")

,utility_id_eia,state,balancing_authority_code_eia,utility_name_eia,water_heater,customer_class,actual_peak_demand_savings_mw,customer_incentives_cost,customers,energy_savings_mwh,other_costs,potential_peak_demand_savings_mw,report_date
9050,84,MD,PJM,A & N Electric Coop,<NA>,commercial,NaN,NaN,<NA>,NaN,NaN,NaN,2017-01-01
9051,84,MD,PJM,A & N Electric Coop,<NA>,industrial,NaN,NaN,<NA>,NaN,NaN,NaN,2017-01-01
9052,84,MD,PJM,A & N Electric Coop,<NA>,residential,0.4,0.0,35,1.0,1000.0,0.5,2017-01-01
9053,84,MD,PJM,A & N Electric Coop,<NA>,total,0.4,0.0,35,1.0,1000.0,0.5,2017-01-01
9054,84,MD,PJM,A & N Electric Coop,<NA>,transportation,NaN,NaN,<NA>,NaN,NaN,NaN,2017-01-01
9055,84,VA,PJM,A & N Electric Coop,<NA>,commercial,NaN,NaN,<NA>,NaN,NaN,NaN,2017-01-01
9056,84,VA,PJM,A & N Electric Coop,<NA>,industrial,NaN,NaN,<NA>,NaN,NaN,NaN,2017-01-01
9057,84,VA,PJM,A & N Electric Coop,<NA>,residential,1.0,0.0,2471,110.0,15000.0,2.0,2017-01-01
9058,84,VA,PJM,A & N Electric Coop,<NA>,total,1.0,0.0,2471,110.0,15000.0,2.0,2017-01-01
9059,84,VA,PJM,A & N Electric Coop,<NA>,transportation,NaN,NaN,<NA>,NaN,NaN,NaN,2017-01-01


In [242]:
dr_merge = pd.merge(dr_tot, dr_group, on=idx_cols, how='outer', suffixes=('_calc_tot', '_og_tot'))

In [243]:
dr_merge

,utility_id_eia,balancing_authority_code_eia,water_heater_calc_tot,customer_class,actual_peak_demand_savings_mw_calc_tot,customer_incentives_cost_calc_tot,customers_calc_tot,energy_savings_mwh_calc_tot,other_costs_calc_tot,potential_peak_demand_savings_mw_calc_tot,report_date,water_heater_og_tot,actual_peak_demand_savings_mw_og_tot,customer_incentives_cost_og_tot,customers_og_tot,energy_savings_mwh_og_tot,other_costs_og_tot,potential_peak_demand_savings_mw_og_tot
0,84,UNK,<NA>,total,1.4,21000.0,2200,100.0,NaN,1.4,2013-01-01,0,1.4,21000.0,2200,100.0,0.0,1.4
1,97,UNK,91,total,3.5,69000.0,4222,NaN,3000.0,8.0,2013-01-01,364,3.5,69000.0,4222,0.0,3000.0,8.0
2,108,UNK,-2,total,24.4,NaN,5139,7747.0,163000.0,76.7,2013-01-01,-8,24.4,0.0,5139,7747.0,163000.0,76.7
3,189,UNK,1718,total,9.4,0.0,11311,0.0,304000.0,9.4,2013-01-01,6872,9.4,0.0,11311,0.0,304000.0,9.4
4,195,UNK,<NA>,total,139.6,99000.0,4061,4884.0,19029000.0,1351.6,2013-01-01,0,139.6,99000.0,4061,4884.0,19029000.0,1351.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138539,60631,WAUW,<NA>,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,2014-01-01,<NA>,NaN,NaN,<NA>,NaN,NaN,NaN
138540,60631,WAUW,<NA>,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,2015-01-01,<NA>,NaN,NaN,<NA>,NaN,NaN,NaN
138541,60631,WAUW,<NA>,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,2016-01-01,<NA>,NaN,NaN,<NA>,NaN,NaN,NaN
138542,60631,WAUW,<NA>,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,2017-01-01,<NA>,NaN,NaN,<NA>,NaN,NaN,NaN


In [176]:
dr_tot = (
    dr_piv.assign(agg_tot=lambda x: x.residential + x.commercial + x.industrial + x.transportation)
)
dr_tot

TypeError: cannot insert an item into a CategoricalIndex that is not already an existing category

# Data Analysis and Visualization
* Now that you've got the required data in a usable form, you can tell the story of your analysis through a mix of visualizations, and further data wrangling steps.
* This narrative should be readable, with figures that have titles, legends, and labeled axes as appropriate so others can understand what you're showing them.
* The code should be concise and make use of the parameters and functions which you've defined above when possible. Functions should contain comprehensible chunks of work that make sense as one step in the story of the analysis.